In [1]:
# from utils import IsingModel1D, IsingModel2D
import numpy as np
from sklearn.neural_network import BernoulliRBM
import pandas as pd
from tqdm import tqdm

In [55]:
# Toy example: https://www.youtube.com/watch?v=Fkw0_aAtwIw&t=141s&ab_channel=DeepLearning.TVDeepLearning.TV
dummy_data = np.random.choice(a=[0,1],p=[0.6,0.4],size=200)
dummy_data = np.array([ (1-a)*np.array([1,0,1]) + (a)*np.array([0,1,0]) for a in dummy_data])

rbm = BernoulliRBM(n_components=2, n_iter=1000, learning_rate=0.01) # 2 hidden units
rbm.fit(dummy_data) # fitting rbm to dummy data generated

def sig(X):
  return (1+np.exp(-X))**-1

v = np.array([0,1,0])

print(sig(rbm.intercept_hidden_[1] + np.dot(v,rbm.components_[1,:]))) # validating correct probability is being computed

# checking the properties of the RBM after learning
print(rbm.transform(v.reshape(1,3)))
print(rbm.gibbs(np.array([[0,1,0],[1,0,1]])).astype(int))

0.0009442872269784528
[[0.00105424 0.00094429]]
[[0 1 0]
 [1 0 1]]


In [56]:
# My own class representation of a deep belief network
class DBN:
  # Initializing RBMs that constitute DBN
  def __init__(self, n_rbms, n_components, learning_rates, n_iter=100, batch_size=10):
    self.rbms = [BernoulliRBM(n_components=n,learning_rate=lr,n_iter=n_iter, batch_size=batch_size) for n,lr in zip(n_components,learning_rates)] # initializing RBMs
    self.n_rbms = n_rbms
    self.n_components = n_components
  # Single backward step for H of a given RBM (row vectors)
  def rbm_backward(self, H, rbm):
    signal = rbm.intercept_visible_.reshape((1, len(rbm.intercept_visible_))) + np.matmul(H,rbm.components_)
    Vp = (1 + np.exp(-signal))**-1
    return np.random.binomial(n=1, p=Vp, size=Vp.shape) # sampled components of the previous layer
  # Network forwarding through each RBM
  def forward(self, V):
    # Make visible unit the 'previous hidden unit' samples
    Hs = V.copy()
    for i in range(self.n_rbms):
      Hp = self.rbms[i].transform(Hs) # get probability distribution over hidden units
      Hs = np.random.binomial(n=1, p=Hp, size=Hp.shape)
    return Hp, Hs # return outputs and probabilities of all hidden layers
  # Network sending signal backwards through each RBM
  def backward(self, H):
    Hs = H
    for i in reversed(range(self.n_rbms)): # iteratively sample backwards until visible units
      Hs = self.rbm_backward(Hs, self.rbms[i])
    return Hs
  # One step forward for a given input, then backward to obtain the corresponding reconstruction
  def reconstruct(self, V):
    if V.ndim == 1: # make 1dim arrays 2d
      V = V.reshape(1,len(V))
    _,Hs = self.forward(V)
    V_reconstruct = self.backward(Hs)
    return V_reconstruct
  # Training all constituents RBMs
  def fit(self, X):
    print('Training DBN Layers ... \n')
    # Make visible unit the 'previous hidden unit' samples
    Hs = X.copy()
    for i in tqdm(range(self.n_rbms)):
      self.rbms[i].fit(Hs)
      Hp = self.rbms[i].transform(Hs) # get probability distribution over hidden units
      Hs = np.random.binomial(n=1, p=Hp, size=Hp.shape) # get sampled hidden units
  def get_weights(self): # return a list of weight matrices for each RBM
    Ws = []
    for i in range(self.n_rbms):
      Ws.append(self.rbms[i].components_)
    return Ws

In [57]:
belief_net = DBN(n_rbms=2, n_components=[2,1], learning_rates=[0.01,0.01, 0.01], n_iter=1000)
belief_net.fit(dummy_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Training DBN Layers ... 



100%|██████████| 2/2 [00:02<00:00,  1.45s/it]


In [73]:
belief_net.reconstruct(np.array([0,1,0]))

array([[0, 1, 0]])

In [32]:
belief_net.get_weights()

[array([[ 0.07493999, -0.09550291,  0.09382636],
        [ 0.07462787, -0.08687521,  0.07220814]]),
 array([[0.17296312, 0.0646226 ]])]

In [17]:
rbm = BernoulliRBM(n_components=2, n_iter=1000, learning_rate=0.01) # 2 hidden units
rbm.fit(dummy_data)

BernoulliRBM(batch_size=10, learning_rate=0.01, n_components=2, n_iter=1000,
             random_state=None, verbose=0)

In [18]:
print(rbm.components_)
print(belief_net.rbms[0].components_)

[[ 0.52900305 -0.53337015  0.52180209]
 [ 0.50327988 -0.50455096  0.48453792]]
[[ 0.46417367 -0.49573166  0.47357885]
 [ 0.49047391 -0.51234901  0.48109665]]
